# Project 1 – Charles – Metrics

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib
import seaborn as sns

# Adding Session ID

In [13]:
# do not run this entire section if your dataset already has unique ID

In [2]:
#data = pd.read_csv("Cleaned_Data.csv", low_memory=False)
#df = data.sort_values(by=['distinct_id', 'time'], axis=0, ascending=True)
#df['ae_session_length'] = df['ae_session_length'].fillna('-1')
#df['ae_session_length'] = df['ae_session_length'].str.replace(',', '')
#df['ae_session_length'] = df['ae_session_length'].astype(float)
#df['time'] = df['time'] = pd.to_datetime(df['time'])
#df

In [3]:
def add_session_id(df):
    df['session_id'] = -1
    sessions = df[['name_x', 'distinct_id', 'time', 'ae_session_length']].loc[df['name_x'] == '$ae_session']
    sessions['start'] = sessions['time'] - np.array([pd.Timedelta(seconds=np.ceil(x)) for x in sessions['ae_session_length']])
    curr_id = 1
    for index, row in sessions.iterrows():
        df.loc[(df['session_id'] == -1) 
               & (df['time'] <= row['time']) 
               & (df['time'] >= row['start']) 
               & (df['distinct_id'] == row['distinct_id']), 'session_id'] = curr_id
        curr_id += 1

#add_session_id(df)
#df
    

In [4]:
#df.loc[df['session_id'] == -1]

In [5]:
#df.to_csv("Cleaned_Dataset_With_ID.csv")

# Adding Time Between Sessions

In [5]:
data = pd.read_csv("Cleaned_Dataset_With_ID.csv", low_memory=False)

In [7]:
data.drop(columns=['Unnamed: 0'], axis=1, inplace=True)


In [8]:
data = data.sort_values(by=['distinct_id', 'time'], axis=0, ascending=True)
data['time'] = pd.to_datetime(data['time'])
data

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id,time_delta
0,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0
1,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0
2,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
3,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
4,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190366,Feed - Scroll Down,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:23,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,0 days 00:03:08
190367,Profile - Change Section,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:28,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,0 days 00:03:08
190368,$ae_session,xbarbarazhong@gmail.com_6704,2019-12-12 14:30:54,190.4,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,0 days 00:03:08
190369,AppOpen,yoon.s.jeong@gmail.com_6485,2019-10-09 11:12:29,-1.0,Chicago,Illinois,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29814,0


In [29]:
grouped = data.loc[data['session_id'] != -1].groupby(['distinct_id', 'session_id'], as_index=False).agg({'time' : ['min', 'max']})
grouped

distinct_id session_id                time  \
                                                                       min   
0      000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD          1 2019-08-23 13:48:32   
1      000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD          2 2019-08-23 13:50:06   
2      000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD          3 2019-08-23 13:50:58   
3      000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD          4 2019-08-23 14:37:20   
4      000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD          5 2019-08-23 15:02:56   
...                                     ...        ...                 ...   
29808               willwojt@gmail.com_6868      29810 2020-03-08 10:52:05   
29809             wolphramite@gmail.com_291      29811 2019-08-20 01:43:37   
29810          xbarbarazhong@gmail.com_6704      29812 2019-12-12 14:26:47   
29811          xbarbarazhong@gmail.com_6704      29813 2019-12-12 14:27:46   
29812           yoon.s.jeong@gmail.com_6485      29814 2019-10-09 11:12:29   

                           
                      max  
0     2019-08-23 13:49:02  
1     2019-08-23 13:50:51  
2     2019-08-23 13:51:52  
3     2019-08-23 14:38:30  
4     2019-08-23 15:05:10  
...                   ...  
29808 2020-03-08 10:52:05  
29809 2019-08-20 01:43:37  
29810 2019-12-12 14:27:43  
29811 2019-12-12 14:30:54  
29812 2019-10-09 11:13:09  

[29813 rows x 4 columns]

In [30]:
grouped.columns

MultiIndex([('distinct_id',    ''),
            ( 'session_id',    ''),
            (       'time', 'min'),
            (       'time', 'max')],
           )

In [46]:
grouped.iloc[0,3] - grouped.iloc[0,2]

Timedelta('0 days 00:00:30')

In [71]:
def add_time_delta(df):
    df['time_delta'] = 0
    grouped = df.loc[df['session_id'] != -1].groupby(['distinct_id', 'session_id'], as_index=False).agg({'time' : ['min', 'max']})
    curr_distinct_id = ""
    for index, row in grouped.iterrows():
        if row[0] != curr_distinct_id:
            #print(row[0])
            curr_distinct_id = row[0]
        else:
            delta = row[3] - row[2]
            #print(delta)
            df.loc[(df['distinct_id'] == row[0]) 
                   & (df['session_id'] == row[1]), 'time_delta'] = delta

add_time_delta(data)
data

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id,time_delta
0,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0
1,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0
2,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
3,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
4,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190366,Feed - Scroll Down,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:23,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,0 days 00:03:08
190367,Profile - Change Section,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:28,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,0 days 00:03:08
190368,$ae_session,xbarbarazhong@gmail.com_6704,2019-12-12 14:30:54,190.4,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,0 days 00:03:08
190369,AppOpen,yoon.s.jeong@gmail.com_6485,2019-10-09 11:12:29,-1.0,Chicago,Illinois,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29814,0


In [69]:
def recover(function):
    import inspect
    get_ipython().set_next_input("".join(inspect.getsourcelines(function)[0]))


In [72]:
data.loc[data['time_delta'] == 0]

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id,time_delta
0,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0
1,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0
2,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
3,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
4,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190349,Sidebar - Go to Feedback,xbarbarazhong@gmail.com_6704,2019-12-12 14:27:39,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29812,0
190350,Visit App Feedback Page,xbarbarazhong@gmail.com_6704,2019-12-12 14:27:39,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29812,0
190351,$ae_session,xbarbarazhong@gmail.com_6704,2019-12-12 14:27:43,57.9,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29812,0
190369,AppOpen,yoon.s.jeong@gmail.com_6485,2019-10-09 11:12:29,-1.0,Chicago,Illinois,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29814,0


In [73]:
data.to_csv("Cleaned_Dataset_With_ID.csv")